# MLP with **TensorFlow/Keras** — Binary Classification

In this notebook you'll train a small **Multi-Layer Perceptron (MLP)** using **TensorFlow/Keras** on a simple 2‑D synthetic dataset.

What you'll learn:
- Building a dataset (two Gaussian blobs) for binary classification  
- Defining an MLP with `tf.keras.Sequential`  
- Compiling with loss/optimizer/metrics and training with `.fit()`  
- Plotting **loss** and **accuracy** curves  
- Visualizing the **decision boundary**  


> **Note:** If `tensorflow` isn't installed in your environment, please install it first.  
> In many environments: `pip install tensorflow`  
> (Skip this if TensorFlow is already available.)

In [1]:
# Imports
import numpy as np
import matplotlib.pyplot as plt

# TensorFlow / Keras
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Reproducibility (best-effort)
np.random.seed(42)
tf.random.set_seed(42)


ModuleNotFoundError: No module named 'matplotlib'

## 1) Create a Simple 2-D Dataset (Binary Classification)

Two clusters (class 0 and class 1). We'll split into train/test.


In [ ]:
def make_blobs(n_per_class=200, spread=1.2):
    mean0 = np.array([-2.0, -2.0])
    mean1 = np.array([ 2.0,  2.0])
    cov = np.array([[spread, 0.0],[0.0, spread]])
    X0 = np.random.multivariate_normal(mean0, cov, n_per_class)
    X1 = np.random.multivariate_normal(mean1, cov, n_per_class)
    X = np.vstack([X0, X1]).astype("float32")
    y = np.hstack([np.zeros(n_per_class), np.ones(n_per_class)]).astype("float32")
    return X, y

X, y = make_blobs(n_per_class=300, spread=1.5)

# Train/test split
indices = np.arange(len(X))
np.random.shuffle(indices)
train_ratio = 0.75
split = int(train_ratio * len(X))
train_idx, test_idx = indices[:split], indices[split:]

X_train, y_train = X[train_idx], y[train_idx]
X_test,  y_test  = X[test_idx],  y[test_idx]

# Quick visualization
plt.figure()
plt.scatter(X_train[:,0], X_train[:,1], s=10)
plt.title("Training Samples (2D points)")
plt.xlabel("x1"); plt.ylabel("x2")
plt.show()


## 2) Define the MLP Model (Keras `Sequential`)

We'll use a small network:

- Dense(16, ReLU) → Dense(8, ReLU) → Dense(1, Sigmoid)

The sigmoid output gives a probability of class **1**.


In [ ]:
model = keras.Sequential([
    layers.Input(shape=(2,)),
    layers.Dense(16, activation='relu'),
    layers.Dense(8, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

model.summary()


## 3) Compile & Train

- **Loss:** `binary_crossentropy`  
- **Optimizer:** `adam`  
- **Metrics:** `accuracy`  
- We'll also track validation performance with `validation_split=0.2`.


In [ ]:
model.compile(
    loss='binary_crossentropy',
    optimizer=keras.optimizers.Adam(learning_rate=0.01),
    metrics=['accuracy'],
)

history = model.fit(
    X_train, y_train,
    epochs=200,
    batch_size=64,
    validation_split=0.2,
    verbose=0  # set 1 to see per-epoch logs
)

# Plot loss
plt.figure()
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='val')
plt.title("Binary Cross-Entropy Loss")
plt.xlabel("Epoch"); plt.ylabel("Loss")
plt.legend()
plt.show()

# Plot accuracy
plt.figure()
plt.plot(history.history['accuracy'], label='train')
plt.plot(history.history['val_accuracy'], label='val')
plt.title("Accuracy")
plt.xlabel("Epoch"); plt.ylabel("Accuracy")
plt.legend()
plt.show()


## 4) Evaluate on the Test Set


In [ ]:
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)
print("Test loss:", round(float(test_loss), 4))
print("Test acc :", round(float(test_acc), 4))


## 5) Visualize the Decision Boundary


In [ ]:
def plot_decision_boundary_keras(model, X, y, steps=200, padding=1.0):
    x_min, x_max = X[:,0].min()-padding, X[:,0].max()+padding
    y_min, y_max = X[:,1].min()-padding, X[:,1].max()+padding
    xx, yy = np.meshgrid(
        np.linspace(x_min, x_max, steps),
        np.linspace(y_min, y_max, steps)
    )
    grid = np.c_[xx.ravel(), yy.ravel()].astype("float32")
    probs = model.predict(grid, verbose=0).reshape(xx.shape)

    plt.figure()
    plt.contourf(xx, yy, probs, levels=20, alpha=0.5)
    plt.scatter(X[:,0], X[:,1], c=y, s=10)
    plt.title("Decision Boundary — TensorFlow MLP")
    plt.xlabel("x1"); plt.ylabel("x2")
    plt.show()

plot_decision_boundary_keras(model, X_train, y_train)


## 6) Tips & Exercises

- Try different widths/depths: e.g., 1 hidden layer with 4 units vs. 2 layers with 32 units.  
- Tune learning rate: `learning_rate=0.001` vs `0.01`.  
- Add regularization: `layers.Dense(16, activation='relu', kernel_regularizer=keras.regularizers.l2(1e-4))`.  
- Switch activation to `tanh` and compare learning curves.  
- Increase overlap between the two blobs to make classification harder.
